In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from utils.diff import diff

In [2]:
# 62.  ps
# 季度频率。9个虚拟变量的和，定义类似于Piotroski(2000)[6]。根据原文的定义，是以下九个指标：
# delta表示本期减去上期：

df1 = pd.read_csv('../../data/csmar/季_资产负债表.csv', usecols=['证券代码', '会计期间', '资产总计', '所有者权益合计', '流动资产合计', '流动负债合计'], low_memory=False)
df2 = pd.read_csv('../../data/csmar/季_利润表.csv', usecols=['证券代码', '会计期间', '净利润', '利润总额', '营业总收入'])
df3 = pd.merge(df1, df2, on=['证券代码', '会计期间'], how='left')
df4 = pd.read_csv('../../data/csmar/季_现金流量表(直接法).csv', usecols=['证券代码', '会计期间', '现金及现金等价物净增加额'])
df5 = pd.merge(df3, df4, on=['证券代码', '会计期间'], how='left')
df5 = diff(df5, col=['净利润', '利润总额', '营业总收入', '现金及现金等价物净增加额'])
df5['年份'] = df5['会计期间'].apply(lambda x: x[:4])
df6 = pd.read_csv('../../data/csmar/日_股本变动文件.csv', usecols=['证券代码', '股本变动日期', '股本变动类型'])
df6 = df6[df6['股本变动类型'].isin([10000, 20000, 2, 30000, 3000, 40000, 50000, 5000, 500, 50, 60, 70000, 9])]
df6['年份'] = df6['股本变动日期'].apply(lambda x: x[:4])
df7 = df6.drop_duplicates(subset=['证券代码', '年份'], keep='last')[['证券代码', '年份', '股本变动类型']]
df8 = pd.merge(df5, df7, on=['证券代码', '年份'], how='left')

# F_ROA，若净利润大于0则为1，否则为0
df8['F_ROA'] = np.where(df8['净利润'] > 0, 1, 0)
df8['F_ROA'][df8['净利润'].isna()] = np.nan
# F_dROA，若delta净利润大于0则为1，否则为0
df8['dROA'] = df8['净利润'] - df8.groupby('证券代码').shift(1)['净利润']
df8['F_dROA'] = np.where(df8['dROA'] > 0, 1, 0)
df8['F_dROA'][df8['dROA'].isna()] = np.nan
# F_CFO，若营运现金流大于0则为1，否则为0
df8['F_CFO'] = np.where(df8['现金及现金等价物净增加额'] > 0, 1, 0)
df8['F_CFO'][df8['现金及现金等价物净增加额'].isna()] = np.nan
# F_ACCRUAL，若营运现金流大于净利润则为1，否则为0
df8['F_ACCRUAL'] = np.where(df8['现金及现金等价物净增加额'] > df8['净利润'],1,0)
df8['F_ACCRUAL'][df8['现金及现金等价物净增加额'].isna()] = np.nan
df8['F_ACCRUAL'][df8['净利润'].isna()] = np.nan
# F_dMARGIN，若d毛利润比（毛利润/总收入）大于0则为1，否则为0
df8['MARGIN'] = df8['利润总额']/df8['营业总收入'].replace(0, np.nan)
df8['dMARGIN'] = df8['MARGIN'] - df8.groupby('证券代码').shift(1)['MARGIN']
df8['F_dMARGIN'] = np.where(df8['dMARGIN'] > 0, 1, 0)
df8['F_dMARGIN'][df8['dMARGIN'].isna()] = np.nan
# F_dTURN，若d资产周转率（总收入/上期总资产）大于0则为1，否则为0
df8['TURN'] = df8['营业总收入']/df8.groupby('证券代码').shift(1)['资产总计'].replace(0, np.nan)
df8['dTURN'] = df8['TURN'] - df8.groupby('证券代码').shift(1)['TURN']
df8['F_dTURN'] = np.where(df8['dTURN'] > 0, 1, 0)
df8['F_dTURN'][df8['dTURN'].isna()] = np.nan
# F_dLEVER，若d杠杆率大于0则为1，否则为0
df8['LEVER'] = df8['所有者权益合计']/df8['资产总计'].replace(0, np.nan)
df8['dLEVER'] = df8['LEVER'] - df8.groupby('证券代码').shift(1)['LEVER']
df8['F_dLEVER'] = np.where(df8['dLEVER'] > 0, 1, 0)
df8['F_dLEVER'][df8['dLEVER'].isna()] = np.nan
# F_dLIQUID，若d流动比率（流动资产/流动负债）大于0则为1，否则为0
df8['LIQUID'] = df8['流动资产合计']/df8['流动负债合计'].replace(0, np.nan)
df8['dLIQUID'] = df8['LIQUID'] - df8.groupby('证券代码').shift(1)['LIQUID']
df8['F_dLIQUID'] = np.where(df8['dLIQUID'] > 0, 1, 0)
df8['F_dLIQUID'][df8['dLIQUID'].isna()] = np.nan
# EQ_OFFER，若该公司在投资组合形成的前一年没有发行普通股则为1，否则为0
df8['EQ_OFFER'] = df8.groupby('证券代码').shift(4)['股本变动类型'].isna().astype('int')

df8['ps'] = df8['F_ROA'] + df8['F_dROA'] + df8['F_CFO'] + df8['F_ACCRUAL'] + df8['F_dMARGIN'] + df8['F_dTURN'] + df8['F_dLEVER'] + df8['F_dLIQUID'] + df8['EQ_OFFER']
df9 = df8[['证券代码', '会计期间', 'ps']]
df9.to_csv('../../output/csmar/季_62.csv', index=False, encoding='utf-8-sig')
df9

,证券代码,会计期间,ps
0,1,1990-03-31,NaN
1,1,1990-06-30,NaN
2,1,1990-09-30,NaN
3,1,1990-12-31,NaN
4,1,1991-03-31,NaN
...,...,...,...
728835,900957,2022-03-31,7.0
728836,900957,2022-06-30,7.0
728837,900957,2022-09-30,5.0
728838,900957,2022-12-31,7.0
